In [1]:
import pandas as pd
from bs4 import BeautifulSoup
# import urllib.request
# import urllib.parse
# import urllib.error
import ssl
import json
# import ast
# import os
from urllib.request import Request, urlopen
import random
import time
import glob

## URLs

##### This section generates all URLs that will be used to scrape song lyrics from genius.com. When adding songs, remove any punctuation and write all lowercase

In [2]:
base_url = 'https://genius.com/'
end_url = '-lyrics'

# list of songs
billie_eilish_songs = ['copycat', 'idontwannabeyouanymore', 'my boy', 'watch', 'party favor',
                       'bellyache', 'ocean eyes', 'hostage', 'and vince staples burn', 
                       'everything i wanted', 'bad guy', 'xanny', 'you should see me in a crown',
                       'all the good girls go to hell', 'wish you were gay', 
                       'when the partys over', '8', 'my strange addiction', 'bury a friend',
                       'ilomilo', 'listen before i go', 'i love you', 'goodbye',
                       'everything i wanted']

# did 'velvet mood' and 'recital' so far
billie_holiday_songs = ['prelude to a kiss', 'when your lover has gone', 
                        'please dont talk about me when im gone', 'nice work if you can get it',
                        'ive got a right to sing the blues', 'whats new', 'i hadnt anyone till you',
                        'everything i have is yours', 'if the moon turns green', 'remember',
                        'autumn in new york', 'my man', 'lover come back to me', 'stormy weather',
                        'yesterdays', 'hes funny that way', 'i cant face the music',
                        'how deep is the ocean', 'what a little moonlight can do', 'i cried for you',
                        # starting top songs
                        'ill be seeing you', 'crazy he calls me', 'easy living', 
                        'what a little moonlight can do', 'strange fruit', 'gloomy sunday',
                        'these foolish things', 'you go to my head', 'all of me', 'good morning heartache',
                        'them there eyes', 'what is this thing called love', 'solitude']

# did 'some gave all' so far
billy_ray_cyrus_songs = ['couldve been me', 'achy breaky heart', 'shes not cryin anymore',
                         'wherm i gonna live', 'these boots are made for walkin', 
                         'someday somewhere somehow', 'never thought id fall in love with you',
                         'aint no good goodbye', 'im so miserable', 'some gave all']

# did 'cold spring harbor' so far
# nocturne is not on this list bc it doesn't have proper lyrics
billy_joel_songs = ['shes got a way', 'you can make me free', 'everybody loves you now', 
                    'why judy why', 'falling of the rain', 'turn around', 'you look so good to me',
                    'tomorrow is today', 'got to begin again']

# did 'billy idol' album so far
billy_idol_songs = ['come on come on', 'white wedding', 'hot in the city', 'dead on arrival',
                    'nobodys business', 'love calling', 'hole in the wall', 'shooting stars',
                    'its so cruel', 'congo man']

# did 'billy_currington' album so far
billy_currington_songs = ['i got a feelin', 'thats just me', 'hangin around', 'off my rocker',
                          'walk a little straighter', 'where the girls are', 'time with you',
                          'when she gets close to me', 'growin up down there', 'next time', 
                          'aint what it used to be']

# did 'tale of the tape' album so far
billy_squier_songs = ['the big beat', 'calley oh', 'rich kid', 'like im lovin you',
                      'who knows what a love can do', 'you should be high love', 
                      'whos your boyfriend', 'the musics all right', 'young girls']

billy_graham_songs = []
lil_nas_x_songs = ['old town road remix']

# create url slugs
billie_eilish_slugs = (['Billie-eilish' + '-' + '-'.join(song.split(' ')) 
                        for song in billie_eilish_songs])
billie_holiday_slugs = (['Billie-holiday' + '-' + '-'.join(song.split(' ')) 
                        for song in billie_holiday_songs])
billy_ray_cyrus_slugs = (['Billy-ray-cyrus' + '-' + '-'.join(song.split(' ')) 
                        for song in billy_ray_cyrus_songs])
billy_joel_slugs = (['Billy-joel' + '-' + '-'.join(song.split(' ')) 
                        for song in billy_joel_songs])
billy_idol_slugs = (['Billy-idol' + '-' + '-'.join(song.split(' ')) 
                        for song in billy_idol_songs])
billy_currington_slugs = (['Billy-currington' + '-' + '-'.join(song.split(' ')) 
                        for song in billy_currington_songs])
billy_squier_slugs = (['Billy-squier' + '-' + '-'.join(song.split(' ')) 
                        for song in billy_squier_songs])
billy_graham_slugs = (['Billy-graham' + '-' + '-'.join(song.split(' ')) 
                        for song in billy_graham_songs])
lil_nas_x_slugs = (['Lil-nas-x' + '-' + '-'.join(song.split(' ')) 
                        for song in lil_nas_x_songs])

# list of all the slugs
slug_list = (billie_eilish_slugs + billie_holiday_slugs + billy_ray_cyrus_slugs
             + billy_joel_slugs + billy_idol_slugs + billy_graham_slugs
             + billy_currington_slugs + billy_squier_slugs
             + lil_nas_x_slugs)

################################################
################ TESTING #######################
####### COMMENT THIS OUT TO DO ALL URLS ########
################################################
# slug_list = (billy_joel_slugs + billy_idol_slugs + billy_graham_slugs
#              + billy_currington_slugs + billy_squier_slugs)

# generate urls
url_list = [base_url + slug + end_url for slug in slug_list]
print('There are ', len(url_list), 'urls in the list')

There are  107 urls in the list


In [3]:
slug_list

['Billie-eilish-copycat',
 'Billie-eilish-idontwannabeyouanymore',
 'Billie-eilish-my-boy',
 'Billie-eilish-watch',
 'Billie-eilish-party-favor',
 'Billie-eilish-bellyache',
 'Billie-eilish-ocean-eyes',
 'Billie-eilish-hostage',
 'Billie-eilish-and-vince-staples-burn',
 'Billie-eilish-everything-i-wanted',
 'Billie-eilish-bad-guy',
 'Billie-eilish-xanny',
 'Billie-eilish-you-should-see-me-in-a-crown',
 'Billie-eilish-all-the-good-girls-go-to-hell',
 'Billie-eilish-wish-you-were-gay',
 'Billie-eilish-when-the-partys-over',
 'Billie-eilish-8',
 'Billie-eilish-my-strange-addiction',
 'Billie-eilish-bury-a-friend',
 'Billie-eilish-ilomilo',
 'Billie-eilish-listen-before-i-go',
 'Billie-eilish-i-love-you',
 'Billie-eilish-goodbye',
 'Billie-eilish-everything-i-wanted',
 'Billie-holiday-prelude-to-a-kiss',
 'Billie-holiday-when-your-lover-has-gone',
 'Billie-holiday-please-dont-talk-about-me-when-im-gone',
 'Billie-holiday-nice-work-if-you-can-get-it',
 'Billie-holiday-ive-got-a-right-to-sin

## Scraping

In [40]:
def scrape_songs(slug_list):
    """This function takes in a list of URL slugs for songs and generates
        URLs for genius.com, scrapes the lyrics and title, and returns a 
        dataframe of [artist, song_id, title, lyrics]"""
    
    counter = 0
    
    # get song_ids from current dataset
    filepath = glob.glob('data/scraped_lyrics.csv')
    lyric_df = pd.read_csv(filepath[0])
    song_ids = lyric_df['song_id'].values
    
    # empty list for scraped lyrics
    scrape_list = []
    
    # iterate through all URLs and scrape 
    for slug in slug_list:
        
        # check if the song is not in the dataset
        if slug not in song_ids:
            
            counter += 1
            print(slug)

            # generate url
            base_url = 'https://genius.com/'
            end_url = '-lyrics'
            url = base_url + slug + end_url

            try:
                print(counter, ' starting ', url)

                # ignore SSL certificate errors
                ctx = ssl.create_default_context()
                ctx.check_hostname = False
                ctx.verify_mode = ssl.CERT_NONE

                # make scrape request appear as browser
                req = Request(url, headers = {'User-Agent' : 'Mozilla/5.0'})
                webpage = urlopen(req).read()
                
                # get html 
                soup = BeautifulSoup(webpage, 'html.parser')
                html = soup.prettify('utf-8')
                song_json = {}
                song_json['lyrics'] = []

                # extract title of the song
                for title in soup.findAll('title'):
                    song_json['title'] = title.text.strip()

                # extract the lyrics of the song
                for div in soup.findAll('div', attrs = {'class': 'lyrics'}):
                    song_json['lyrics'].append(div.text.strip().split('\n'))

                # append song id for later reference
                song_json['song_id'] = slug

                # append JSON to list 
                scrape_list.append(song_json)

                # pause time to not get scraper blocked
                time.sleep(random.randint(1,5))
            
            # when there's an error with the url
            except:
                print('error with ', url)
                continue
        
        # song is in current dataset
        else:
            print('skipped duplicate:', slug)
            continue
    
    # create dataframe of scraped songs
    df = pd.DataFrame(scrape_list)
    print(df.shape)
    
    # if new data scraped
    if df.shape[0] != 0: 

        # fix string formatting issue
        # NOTE: the '–' character used in titles column is not a normal dash
        # I had to print a title out and use that dash for it to recognize the 
        # character. Replacing it with a normal - to avoid dealing with it.
        # They look the same but they're not ¯\_(ツ)_/¯
        df['title'] = [title.replace('–','-') for title in df['title']]

        # add artist column
        df['artist'] = [title.split('-')[0] for title in df['title']]

        # clean title column
        df['title'] = [title.replace('Lyrics','-') for title in df['title']]
        df['title'] = [title.split('-')[1] for title in df['title']]
        
        # turn lyrics column into a string
        df['lyrics'] = [str(lyrics[0]) for lyrics in df['lyrics']]
        
        # reorder columns
        df = df[['artist', 'title', 'song_id', 'lyrics']]

        # concatenate with current data
        df = pd.concat([lyric_df, df])
        
    # if no new data scraped    
    else:
        df = lyric_df
        print('no new songs added')
    
    print('shape before dup dropped:', df.shape)
    df = df.drop_duplicates()
    print('shape after dup dropped:', df.shape)
    
    # fix index numbering
    df.reset_index(inplace=True)
    df.drop(columns='index', inplace=True)
    
    return df

df = scrape_songs(slug_list)

skipped duplicate: Billie-eilish-copycat
skipped duplicate: Billie-eilish-idontwannabeyouanymore
skipped duplicate: Billie-eilish-my-boy
skipped duplicate: Billie-eilish-watch
skipped duplicate: Billie-eilish-party-favor
skipped duplicate: Billie-eilish-bellyache
skipped duplicate: Billie-eilish-ocean-eyes
skipped duplicate: Billie-eilish-hostage
skipped duplicate: Billie-eilish-and-vince-staples-burn
skipped duplicate: Billie-eilish-everything-i-wanted
skipped duplicate: Billie-eilish-bad-guy
skipped duplicate: Billie-eilish-xanny
skipped duplicate: Billie-eilish-you-should-see-me-in-a-crown
skipped duplicate: Billie-eilish-all-the-good-girls-go-to-hell
skipped duplicate: Billie-eilish-wish-you-were-gay
skipped duplicate: Billie-eilish-when-the-partys-over
skipped duplicate: Billie-eilish-8
skipped duplicate: Billie-eilish-my-strange-addiction
skipped duplicate: Billie-eilish-bury-a-friend
skipped duplicate: Billie-eilish-ilomilo
Billie-eilish-listen-before-i-go
1  starting  https://g

In [42]:
df

,artist,title,song_id,lyrics
0,Billie Eilish,COPYCAT,Billie-eilish-copycat,"Don't be cautious, don't be kind, You committ..."
1,Billie Eilish,​idontwannabeyouanymore,Billie-eilish-idontwannabeyouanymore,"Don't be that way, 'Fall apart twice a day', ..."
2,Billie Eilish,​my boy,Billie-eilish-my-boy,"'(Three and four and)', 'Ba-ba-da, ba-ba-da-b..."
3,Billie Eilish,​watch,Billie-eilish-watch,"'Lips meet teeth and tongue', 'My heart skips..."
4,Billie Eilish,​​party favor,Billie-eilish-party-favor,"'Hey, leave a message', 'Hey, call me back ..."
5,Billie Eilish,​bellyache,Billie-eilish-bellyache,"'Mind', 'Mind', Sittin' all alone, 'Mouth f..."
6,Billie Eilish,​​ocean eyes,Billie-eilish-ocean-eyes,"I've been watchin' you for some time, Can't s..."
7,Billie Eilish,​hostage,Billie-eilish-hostage,"'I wanna be alone', 'Alone with you, does tha..."
8,Billie Eilish & Vince Staples,​&burn,Billie-eilish-and-vince-staples-burn,"': ', 'Lips meet teeth and tongue', My heart s..."
9,Billie Eilish,​everything i wanted,Billie-eilish-everything-i-wanted,"'I had a dream', 'I got everything I wanted',..."


In [43]:
print(type(df.lyrics.loc[8]))

<class 'str'>


In [28]:
# df = df.drop([0,1,2,3,4,5])
df

,artist,title,song_id,lyrics
0,Billie Eilish,COPYCAT,Billie-eilish-copycat,"Don't be cautious, don't be kind, You committ..."
1,Billie Eilish,​idontwannabeyouanymore,Billie-eilish-idontwannabeyouanymore,"Don't be that way, 'Fall apart twice a day', ..."
2,Billie Eilish,​my boy,Billie-eilish-my-boy,"'(Three and four and)', 'Ba-ba-da, ba-ba-da-b..."
3,Billie Eilish,​watch,Billie-eilish-watch,"'Lips meet teeth and tongue', 'My heart skips..."
4,Billie Eilish,​​party favor,Billie-eilish-party-favor,"'Hey, leave a message', 'Hey, call me back ..."
5,Billie Eilish,​bellyache,Billie-eilish-bellyache,"'Mind', 'Mind', Sittin' all alone, 'Mouth f..."
6,Billie Eilish,​​ocean eyes,Billie-eilish-ocean-eyes,"I've been watchin' you for some time, Can't s..."
7,Billie Eilish,​hostage,Billie-eilish-hostage,"'I wanna be alone', 'Alone with you, does tha..."
8,Billie Eilish & Vince Staples,​&burn,Billie-eilish-and-vince-staples-burn,"': ', 'Lips meet teeth and tongue', My heart s..."
9,Billie Eilish,​everything i wanted,Billie-eilish-everything-i-wanted,"'I had a dream', 'I got everything I wanted',..."


In [45]:
def clean_lyrics(df):
    """This function takes a dataframe of lyrics and removes any unwanted
       characters and returns the dataframe"""
    
    df = df.copy()

    # custom words and characters to remove
    stop_words = ['Verse 1', 'Verse 2', 'Verse 3', 'Verse 4', 'Chorus', 
                  'Bridge', ',,', 'Intro:', 'Intro',
                  'Billie Eilish', 'Billie Holiday', 'Billy Rae Cyrus',
                  'Billy Joel', 'Billy Idol', 'Billy Graham', 
                  'Vince Staples',
                  '\\n', '{', '}', '[', ']', 'lyrics', 'title', '"', 'genius', "'',"]
    # text = text.replace("\", '')
    
    for word in stop_words:
        df['lyrics'] = [lyrics.replace(word, '') for lyrics in df['lyrics']]
        
    return df

In [46]:
clean_df = clean_lyrics(df)
clean_df

,artist,title,song_id,lyrics
0,Billie Eilish,COPYCAT,Billie-eilish-copycat,"Don't be cautious, don't be kind, You committ..."
1,Billie Eilish,​idontwannabeyouanymore,Billie-eilish-idontwannabeyouanymore,"Don't be that way, 'Fall apart twice a day', ..."
2,Billie Eilish,​my boy,Billie-eilish-my-boy,"'(Three and four and)', 'Ba-ba-da, ba-ba-da-b..."
3,Billie Eilish,​watch,Billie-eilish-watch,"'Lips meet teeth and tongue', 'My heart skips..."
4,Billie Eilish,​​party favor,Billie-eilish-party-favor,"'Hey, leave a message', 'Hey, call me back ..."
5,Billie Eilish,​bellyache,Billie-eilish-bellyache,"'Mind', 'Mind', Sittin' all alone, 'Mouth f..."
6,Billie Eilish,​​ocean eyes,Billie-eilish-ocean-eyes,"I've been watchin' you for some time, Can't s..."
7,Billie Eilish,​hostage,Billie-eilish-hostage,"'I wanna be alone', 'Alone with you, does tha..."
8,Billie Eilish & Vince Staples,​&burn,Billie-eilish-and-vince-staples-burn,"': ', 'Lips meet teeth and tongue', My heart s..."
9,Billie Eilish,​everything i wanted,Billie-eilish-everything-i-wanted,"'I had a dream', 'I got everything I wanted',..."


## Testing

In [36]:
print(clean_df.lyrics.loc[8])
print(type(clean_df.lyrics.loc[8]))

': ', 'Lips meet teeth and tongue', My heart skips eight beats at once (That's better), 'If we were meant to be, we would have been by now', 'See what you wanna see, all I see is him right now', 'H-h-him right now',  ': ', I'll sit and watch your car burn, 'With the fire that you started in me', 'But you never came back to ask it out', 'Go ahead and watch my heart burn', 'With the fire that you started in me', But I'll never let you back to put it out, '(Thanks)',  ': ', 'Your love feels so fake', My demands aren't high to make, 'If I could get to sleep, I would have slept by now (Ahh)', 'Your lies will never keep, I think you need to blow them out', '(B-b-blow them out)',  ': ', I'll sit and watch your car burn, 'With the fire that you started in me', 'But you never came back to ask it out', 'Go ahead and watch my heart burn', 'With the fire that you started in me', But I'll never let you back to put it out,  ': ', '7-4-2008', 'I still remember that', 'Heaven sent a present my way', I

## Export

In [47]:
clean_df.to_csv('data/scraped_lyrics.csv', index=False)

In [48]:
import glob
filepath = glob.glob('data/scraped_lyrics.csv')
print(filepath)
pd.read_csv(filepath[0])
# pd.read_csv('scraped_lyrics.csv')

['data/scraped_lyrics.csv']


,artist,title,song_id,lyrics
0,Billie Eilish,COPYCAT,Billie-eilish-copycat,"Don't be cautious, don't be kind, You committ..."
1,Billie Eilish,​idontwannabeyouanymore,Billie-eilish-idontwannabeyouanymore,"Don't be that way, 'Fall apart twice a day', ..."
2,Billie Eilish,​my boy,Billie-eilish-my-boy,"'(Three and four and)', 'Ba-ba-da, ba-ba-da-b..."
3,Billie Eilish,​watch,Billie-eilish-watch,"'Lips meet teeth and tongue', 'My heart skips..."
4,Billie Eilish,​​party favor,Billie-eilish-party-favor,"'Hey, leave a message', 'Hey, call me back ..."
5,Billie Eilish,​bellyache,Billie-eilish-bellyache,"'Mind', 'Mind', Sittin' all alone, 'Mouth f..."
6,Billie Eilish,​​ocean eyes,Billie-eilish-ocean-eyes,"I've been watchin' you for some time, Can't s..."
7,Billie Eilish,​hostage,Billie-eilish-hostage,"'I wanna be alone', 'Alone with you, does tha..."
8,Billie Eilish & Vince Staples,​&burn,Billie-eilish-and-vince-staples-burn,"': ', 'Lips meet teeth and tongue', My heart s..."
9,Billie Eilish,​everything i wanted,Billie-eilish-everything-i-wanted,"'I had a dream', 'I got everything I wanted',..."
